In [6]:
import cv2
import numpy as np

def process_cloud_chamber_video(video_path, start_frame=0, end_frame=None):
    # Load the video
    cap = cv2.VideoCapture(video_path)
    
    # Get total frame count and set end frame if not specified
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    end_frame = end_frame or total_frames
    
    # Set video position to the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    # Read the first frame to initialize count matrix
    ret, first_frame = cap.read()
    height, width = first_frame.shape[:2]
    count_matrix = np.zeros((height, width), dtype=np.int32)
    
    # Process each frame from start to end
    for frame_idx in range(start_frame, end_frame):
        if not ret:
            break
        
        gray_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
        _, binary_frame = cv2.threshold(gray_frame, 100, 1, cv2.THRESH_BINARY)
        count_matrix += binary_frame
        
        ret, first_frame = cap.read()
    
    cap.release()
    
    # Apply logarithmic scale to the count matrix
    count_matrix_log = np.log1p(count_matrix)
    max_log_count = np.max(count_matrix_log)
    
    # Normalize to range 0-5
    norm_matrix = ((count_matrix_log / max_log_count) * 5).astype(np.uint8)
    
    # Define the color map for 5 discrete values
    colors = {
        0: (255, 0, 0),    # Blue
        1: (0, 255, 0),    # Green
        2: (0, 255, 255),  # Yellow
        3: (0, 165, 255),  # Orange
        4: (0, 0, 255),    # Red
        5: (255, 0, 0)     # Blue (again, for max values)
    }
    
    # Generate the output image
    output_image = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(6):
        output_image[norm_matrix == i] = colors[i]
    
    # Save the output image
    output_filename = f'output_image_{start_frame}_to_{end_frame}.png'
    cv2.imwrite(output_filename, output_image)
    print(f"Output image saved as {output_filename}")

video_path = r"C:\Users\598Jo\Desktop\detected_motion_short.mp4"
process_cloud_chamber_video(video_path, start_frame=500, end_frame=10000)


Output image saved as output_image_500_to_10000.png
